Mucky notebook to pull annotated motifs from ms2lda.org

In [1]:
import requests,csv

Run one of the three cells below, depending on the experiment of choice

In [9]:
experiment_id = 93 # which experiment
out_path = '/Users/simon/git/motifdb/motifs/urine_converted_to_005/' # where to save the m2m files
prefix = 'urine' # what to append to the name (i.e. to avoid lots of things called motif_x.m2m)
description = '#COMMENT Derived from Urine ms2lda experiment id={}'.format(experiment_id)

In [10]:
url = 'http://ms2lda.org/basicviz/get_annotated_topics/{}'.format(experiment_id)
response = requests.get(url)

In [11]:
def round_mass(mass,bin_width = 0.005):
    # function to find the 0.005 bin for any mass
    import numpy as np
    bins_from_zero = mass/bin_width
    integer_bins = np.floor(bins_from_zero)
    lower_edge = integer_bins * bin_width
    upper_edge = lower_edge + bin_width
    center = (lower_edge + upper_edge)/2.0
    return "{:.4f}".format(center),lower_edge,upper_edge

In [12]:
annotations = {}
spec = {}

for name,annotation,short_annotation in response.json()[0]:
    annotations[name] = (annotation,short_annotation)
for name,s in response.json()[1]:
    spec[name] = {}
    for f,i in s:
        tokens = f.split('_')
        m = float(tokens[1])
        sm,_,_ = round_mass(m)
        new_feature = "{}_{}".format(tokens[0],sm)
        spec[name][new_feature] = i

In [13]:
for name,(annotation,short_annotation) in annotations.items():
    filename = out_path + prefix + '_' + name + '.m2m'
    with open(filename,'w') as f:
        writer = csv.writer(f,delimiter = ',',dialect='excel')
        writer.writerow(['#NAME ' + prefix + '_' +name])
        writer.writerow(['#ANNOTATION ' + " ".join(annotation.encode('utf8').split(','))])
        writer.writerow(['#SHORT_ANNOTATION ' + " ".join(short_annotation.encode('utf8').split(','))])
        writer.writerow([description])
        s = zip(spec[name].keys(),spec[name].values())
        s = sorted(s,key = lambda x: x[1],reverse = True)
        for f,i in s:
            writer.writerow([f,i])